# 1. Skip-gram with naiive softmax
- Implementation model 'Skip-gram with naiive softmax'

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter

In [2]:
flatten = lambda l: [item for sublist in l for item in sublist]
random.seed(1024)

In [3]:
# Replacement of lambda 'flatten'
co = []
for a1 in corpus:
    for a2 in a1:
        co.append(a2)
cor = list(set(co))

NameError: name 'corpus' is not defined

In [13]:
print(torch.__version__)
print(nltk.__version__)

0.4.1
3.2.4


In [14]:
USE_CUDA = torch.cuda.is_available()
USE_CUDA
gpus = [0]
torch.cuda.set_device(gpus[0])

RuntimeError: cuda runtime error (35) : CUDA driver version is insufficient for CUDA runtime version at torch/csrc/cuda/Module.cpp:32

In [15]:
if USE_CUDA:
    FloatTensor = torch.cuda.FloatTensor 
else:
    FloatTensor = torch.FloatTensor
# Upper code can be converted more simple
FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor

In [16]:
def getBatch(batch_size, train_data):
    random.shuffle(train_data)
    sindex = 0
    eindex = batch_size
    while eindex < len(train_data):
        batch = train_data[sindex : eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch
        
    if eindex >= len(train_data):
        batch = train_data[index]
        yield batch

In [17]:
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w : word2index[w] if word2index.get(w) is not None else word2index['<UNK'], seq))

In [48]:
def prepare_word(word, word2index):
    return Variable(LongTensor([word2index[word]]) if word2index.get(word) is not None else LongTensor([word2index['<UNK>']]))
def prepare_word(word, word2index):
    return Variable(LongTensor([word2index[word]]) if word2index.get(word) is not None else LongTensor([word2index["<UNK>"]]))

## 1. Data Load
- Data : Gutenberg corpus

In [19]:
nltk.corpus.gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [20]:
corpus = list(nltk.corpus.gutenberg.sents('melville-moby_dick.txt')) # 텍스트를 scentence화
corpus = [[word.lower() for word in sent] for sent in corpus] # 단어들 소문자화

## 2. Extract stopwords from unigram dstribution's tail

In [21]:
word_count = Counter(flatten(corpus))
word_count

Counter({'[': 3,
         'moby': 84,
         'dick': 84,
         'by': 1204,
         'herman': 1,
         'melville': 1,
         '1851': 3,
         ']': 1,
         'etymology': 1,
         '.': 6880,
         '(': 210,
         'supplied': 12,
         'a': 4736,
         'late': 30,
         'consumptive': 1,
         'usher': 2,
         'to': 4625,
         'grammar': 2,
         'school': 10,
         ')': 78,
         'the': 14431,
         'pale': 19,
         '--': 1090,
         'threadbare': 1,
         'in': 4172,
         'coat': 28,
         ',': 18713,
         'heart': 91,
         'body': 110,
         'and': 6430,
         'brain': 37,
         ';': 4072,
         'i': 2127,
         'see': 272,
         'him': 1067,
         'now': 785,
         'he': 1896,
         'was': 1644,
         'ever': 206,
         'dusting': 2,
         'his': 2530,
         'old': 450,
         'lexicons': 1,
         'grammars': 2,
         'with': 1722,
         'queer': 44,
    

In [22]:
border = int(len(word_count) * 0.01)
border

172

In [23]:
word_count.most_common()[:border]

[(',', 18713),
 ('the', 14431),
 ('.', 6880),
 ('of', 6609),
 ('and', 6430),
 ('a', 4736),
 ('to', 4625),
 ('in', 4172),
 (';', 4072),
 ('that', 3085),
 ("'", 2684),
 ('-', 2552),
 ('his', 2530),
 ('it', 2522),
 ('i', 2127),
 ('he', 1896),
 ('but', 1818),
 ('s', 1802),
 ('as', 1741),
 ('is', 1725),
 ('with', 1722),
 ('was', 1644),
 ('for', 1617),
 ('all', 1526),
 ('"', 1478),
 ('this', 1394),
 ('at', 1319),
 ('!', 1270),
 ('whale', 1226),
 ('by', 1204),
 ('not', 1151),
 ('--', 1090),
 ('from', 1088),
 ('him', 1067),
 ('so', 1065),
 ('on', 1062),
 ('be', 1045),
 ('one', 921),
 ('you', 894),
 ('there', 869),
 ('now', 785),
 ('had', 779),
 ('have', 768),
 ('or', 713),
 ('were', 684),
 ('they', 667),
 ('which', 648),
 ('like', 647),
 ('?', 637),
 ('me', 633),
 ('then', 630),
 ('their', 620),
 ('what', 618),
 ('some', 618),
 ('when', 606),
 ('are', 598),
 ('an', 596),
 ('my', 589),
 ('no', 586),
 ('upon', 566),
 ('out', 538),
 ('man', 527),
 ('into', 523),
 ('up', 521),
 ('ship', 518),
 ('a

In [24]:
list(reversed(word_count.most_common()))[:border]

[('orphan', 1),
 ('retracing', 1),
 ('sheathed', 1),
 ('padlocks', 1),
 ('dirgelike', 1),
 ('liberated', 1),
 ('ixion', 1),
 ('suction', 1),
 ('halfspent', 1),
 ('epilogue', 1),
 ('thrill', 1),
 ('etherial', 1),
 ('intercept', 1),
 ('incommoding', 1),
 ('tauntingly', 1),
 ('backwardly', 1),
 ('coincidings', 1),
 ('ironical', 1),
 ('intermixingly', 1),
 ('whelmings', 1),
 ('inanimate', 1),
 ('animate', 1),
 ('lookouts', 1),
 ('infatuation', 1),
 ('morgana', 1),
 ('fata', 1),
 ('gaseous', 1),
 ('mediums', 1),
 ('bewildering', 1),
 ('bowstring', 1),
 ('mutes', 1),
 ('voicelessly', 1),
 ('grapple', 1),
 ('unconquering', 1),
 ('comber', 1),
 ('foregone', 1),
 ('bullied', 1),
 ('uncracked', 1),
 ('unsurrendered', 1),
 ('flume', 1),
 ('dislodged', 1),
 ('buttress', 1),
 ('predestinating', 1),
 ('inactive', 1),
 ('coppers', 1),
 ('gulping', 1),
 ('assassins', 1),
 ('brushwood', 1),
 ('mattrass', 1),
 ('unwinking', 1),
 ('unappeasable', 1),
 ('fidelities', 1),
 ('plaid', 1),
 ('gap', 1),
 ('spl

In [25]:
stopwords = word_count.most_common()[:border] + list(reversed(word_count.most_common()))[:border]
stopwords

[(',', 18713),
 ('the', 14431),
 ('.', 6880),
 ('of', 6609),
 ('and', 6430),
 ('a', 4736),
 ('to', 4625),
 ('in', 4172),
 (';', 4072),
 ('that', 3085),
 ("'", 2684),
 ('-', 2552),
 ('his', 2530),
 ('it', 2522),
 ('i', 2127),
 ('he', 1896),
 ('but', 1818),
 ('s', 1802),
 ('as', 1741),
 ('is', 1725),
 ('with', 1722),
 ('was', 1644),
 ('for', 1617),
 ('all', 1526),
 ('"', 1478),
 ('this', 1394),
 ('at', 1319),
 ('!', 1270),
 ('whale', 1226),
 ('by', 1204),
 ('not', 1151),
 ('--', 1090),
 ('from', 1088),
 ('him', 1067),
 ('so', 1065),
 ('on', 1062),
 ('be', 1045),
 ('one', 921),
 ('you', 894),
 ('there', 869),
 ('now', 785),
 ('had', 779),
 ('have', 768),
 ('or', 713),
 ('were', 684),
 ('they', 667),
 ('which', 648),
 ('like', 647),
 ('?', 637),
 ('me', 633),
 ('then', 630),
 ('their', 620),
 ('what', 618),
 ('some', 618),
 ('when', 606),
 ('are', 598),
 ('an', 596),
 ('my', 589),
 ('no', 586),
 ('upon', 566),
 ('out', 538),
 ('man', 527),
 ('into', 523),
 ('up', 521),
 ('ship', 518),
 ('a

In [26]:
stopwords = [s[0] for s in stopwords]

In [27]:
stopwords

[',',
 'the',
 '.',
 'of',
 'and',
 'a',
 'to',
 'in',
 ';',
 'that',
 "'",
 '-',
 'his',
 'it',
 'i',
 'he',
 'but',
 's',
 'as',
 'is',
 'with',
 'was',
 'for',
 'all',
 '"',
 'this',
 'at',
 '!',
 'whale',
 'by',
 'not',
 '--',
 'from',
 'him',
 'so',
 'on',
 'be',
 'one',
 'you',
 'there',
 'now',
 'had',
 'have',
 'or',
 'were',
 'they',
 'which',
 'like',
 '?',
 'me',
 'then',
 'their',
 'what',
 'some',
 'when',
 'are',
 'an',
 'my',
 'no',
 'upon',
 'out',
 'man',
 'into',
 'up',
 'ship',
 'ahab',
 'more',
 '."',
 'if',
 'them',
 'ye',
 'we',
 'sea',
 'old',
 'would',
 'other',
 'been',
 'over',
 'these',
 'will',
 'though',
 'down',
 'its',
 'only',
 'such',
 'who',
 'head',
 'yet',
 'boat',
 'time',
 'long',
 'her',
 'captain',
 'any',
 'here',
 'very',
 '!"',
 'still',
 'about',
 'than',
 'do',
 'great',
 'those',
 'said',
 ',"',
 'before',
 'two',
 'has',
 't',
 'must',
 'seemed',
 'most',
 'white',
 'last',
 'see',
 'thou',
 'way',
 'after',
 'whales',
 'again',
 'did',
 '

## 3. Build vocab

In [28]:
len(set(flatten(corpus)))
len(set(stopwords))

344

In [29]:
vocab = list(set(flatten(corpus)) - set(stopwords))
vocab.append('<UNK>')

In [30]:
print(len(set(flatten(corpus))), len(vocab))

17225 16882


In [31]:
word2index = {'<UNK>' : 0}

for vo in vocab:
    if word2index.get(vo) is None:
        word2index[vo] = len(word2index)
        
index2word = {v:k for k, v in word2index.items()}
index2word

{0: '<UNK>',
 1: 'shan',
 2: 'bunched',
 3: 'shone',
 4: 'modifies',
 5: 'particulars',
 6: 'damply',
 7: 'emigrant',
 8: 'stormed',
 9: 'several',
 10: 'lowly',
 11: 'nap',
 12: '127',
 13: 'eternally',
 14: 'ale',
 15: 'dabs',
 16: 'haggardly',
 17: '63',
 18: 'indignant',
 19: 'mind',
 20: 'rifle',
 21: 'exhibiting',
 22: 'live',
 23: 'consorted',
 24: 'inkstand',
 25: 'flame',
 26: '....',
 27: 'pitiable',
 28: 'lurid',
 29: 'punch',
 30: 'second',
 31: 'surprising',
 32: 'puissant',
 33: 'genteel',
 34: 'lodged',
 35: 'thickens',
 36: 'owns',
 37: 'falsely',
 38: 'blight',
 39: 'houses',
 40: 'sentinelled',
 41: 'helpless',
 42: 'partake',
 43: 'desolate',
 44: 'squaring',
 45: 'guineas',
 46: 'enticing',
 47: 'fallacious',
 48: 'pomp',
 49: 'trembles',
 50: 'prosecution',
 51: '1807',
 52: 'shifting',
 53: 'conquered',
 54: 'pliny',
 55: 'underwent',
 56: 'genealogy',
 57: 'forewarned',
 58: 'allurings',
 59: 'talons',
 60: 'scenery',
 61: 'grieve',
 62: 'canes',
 63: 'sneaks',
 

## 4. Prepare train data

In [32]:
WINDOW_SIZE = 3
windows = flatten([list(nltk.ngrams(['<DUMMY>'] * WINDOW_SIZE + c + ['<DUMMY>'] * WINDOW_SIZE, WINDOW_SIZE * 2 + 1)) for c in corpus])
windows

[('<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by'),
 ('<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by', 'herman'),
 ('<DUMMY>', '[', 'moby', 'dick', 'by', 'herman', 'melville'),
 ('[', 'moby', 'dick', 'by', 'herman', 'melville', '1851'),
 ('moby', 'dick', 'by', 'herman', 'melville', '1851', ']'),
 ('dick', 'by', 'herman', 'melville', '1851', ']', '<DUMMY>'),
 ('by', 'herman', 'melville', '1851', ']', '<DUMMY>', '<DUMMY>'),
 ('herman', 'melville', '1851', ']', '<DUMMY>', '<DUMMY>', '<DUMMY>'),
 ('<DUMMY>', '<DUMMY>', '<DUMMY>', 'etymology', '.', '<DUMMY>', '<DUMMY>'),
 ('<DUMMY>', '<DUMMY>', 'etymology', '.', '<DUMMY>', '<DUMMY>', '<DUMMY>'),
 ('<DUMMY>', '<DUMMY>', '<DUMMY>', '(', 'supplied', 'by', 'a'),
 ('<DUMMY>', '<DUMMY>', '(', 'supplied', 'by', 'a', 'late'),
 ('<DUMMY>', '(', 'supplied', 'by', 'a', 'late', 'consumptive'),
 ('(', 'supplied', 'by', 'a', 'late', 'consumptive', 'usher'),
 ('supplied', 'by', 'a', 'late', 'consumptive', 'usher', 'to'),
 ('by', 'a', 'late', 'c

In [33]:
train_data = []

for window in windows:
    for i in range(WINDOW_SIZE * 2 + 1):
        if i == WINDOW_SIZE or window[i] == '<DUMMY>':
            continue
        train_data.append((window[WINDOW_SIZE], window[i]))
train_data

[('[', 'moby'),
 ('[', 'dick'),
 ('[', 'by'),
 ('moby', '['),
 ('moby', 'dick'),
 ('moby', 'by'),
 ('moby', 'herman'),
 ('dick', '['),
 ('dick', 'moby'),
 ('dick', 'by'),
 ('dick', 'herman'),
 ('dick', 'melville'),
 ('by', '['),
 ('by', 'moby'),
 ('by', 'dick'),
 ('by', 'herman'),
 ('by', 'melville'),
 ('by', '1851'),
 ('herman', 'moby'),
 ('herman', 'dick'),
 ('herman', 'by'),
 ('herman', 'melville'),
 ('herman', '1851'),
 ('herman', ']'),
 ('melville', 'dick'),
 ('melville', 'by'),
 ('melville', 'herman'),
 ('melville', '1851'),
 ('melville', ']'),
 ('1851', 'by'),
 ('1851', 'herman'),
 ('1851', 'melville'),
 ('1851', ']'),
 (']', 'herman'),
 (']', 'melville'),
 (']', '1851'),
 ('etymology', '.'),
 ('.', 'etymology'),
 ('(', 'supplied'),
 ('(', 'by'),
 ('(', 'a'),
 ('supplied', '('),
 ('supplied', 'by'),
 ('supplied', 'a'),
 ('supplied', 'late'),
 ('by', '('),
 ('by', 'supplied'),
 ('by', 'a'),
 ('by', 'late'),
 ('by', 'consumptive'),
 ('a', '('),
 ('a', 'supplied'),
 ('a', 'by'),
 (

In [34]:
X_p = []
y_p = []

In [35]:
for tr in train_data:
    X_p.append(prepare_word(tr[0], word2index).view(1, -1))
    y_p.append(prepare_word(tr[1], word2index).view(1, -1))

In [36]:
train_data = list(zip(X_p, y_p))

In [37]:
a = [1,2,3]
b = [4,5,6]
a+ b

[1, 2, 3, 4, 5, 6]

In [39]:
class Skipgram(nn.Module):
    
    def __init__(self, vocab_size, projection_dim):
        super(Skipgram,self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, projection_dim)
        self.embedding_u = nn.Embedding(vocab_size, projection_dim)

        self.embedding_v.weight.data.uniform_(-1, 1) # init
        self.embedding_u.weight.data.uniform_(0, 0) # init
        #self.out = nn.Linear(projection_dim,vocab_size)
    def forward(self, center_words,target_words, outer_words):
        center_embeds = self.embedding_v(center_words) # B x 1 x D
        target_embeds = self.embedding_u(target_words) # B x 1 x D
        outer_embeds = self.embedding_u(outer_words) # B x V x D
        
        scores = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # Bx1xD * BxDx1 => Bx1
        norm_scores = outer_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # BxVxD * BxDx1 => BxV
        
        nll = -torch.mean(torch.log(torch.exp(scores)/torch.sum(torch.exp(norm_scores), 1).unsqueeze(1))) # log-softmax
        
        return nll # negative log likelihood
    
    def prediction(self, inputs):
        embeds = self.embedding_v(inputs)
        
        return embeds 

## 5. Train

In [40]:
EMBEDDING_SIZE = 30
BATCH_SIZE = 256
EPOCH = 100

In [ ]:
losses = []
model = Skipgram(len(word2index), EMBEDDING_SIZE)
if USE_CUDA:
    model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [44]:
model
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)

In [45]:
for epoch in range(EPOCH):
    for i, batch in enumerate(getBatch(BATCH_SIZE, train_data)):
        
        inputs, targets = zip(*batch)
        
        inputs = torch.cat(inputs) # B x 1
        targets = torch.cat(targets) # B x 1
        vocabs = prepare_sequence(list(vocab), word2index).expand(inputs.size(0), len(vocab))  # B x V
        model.zero_grad()

        loss = model(inputs, targets, vocabs)
        
        loss.backward()
        optimizer.step()
   
        losses.append(loss.data.tolist()[0])

    if epoch % 10 == 0:
        print("Epoch : %d, mean_loss : %.02f" % (epoch,np.mean(losses)))
        losses = []

AttributeError: 'NoneType' object has no attribute 'expand'

In [50]:
vocab

['shan',
 'bunched',
 'shone',
 'modifies',
 'particulars',
 'damply',
 'emigrant',
 'stormed',
 'several',
 'lowly',
 'nap',
 '127',
 'eternally',
 'ale',
 'dabs',
 'haggardly',
 '63',
 'indignant',
 'mind',
 'rifle',
 'exhibiting',
 'live',
 'consorted',
 'inkstand',
 'flame',
 '....',
 'pitiable',
 'lurid',
 'punch',
 'second',
 'surprising',
 'puissant',
 'genteel',
 'lodged',
 'thickens',
 'owns',
 'falsely',
 'blight',
 'houses',
 'sentinelled',
 'helpless',
 'partake',
 'desolate',
 'squaring',
 'guineas',
 'enticing',
 'fallacious',
 'pomp',
 'trembles',
 'prosecution',
 '1807',
 'shifting',
 'conquered',
 'pliny',
 'underwent',
 'genealogy',
 'forewarned',
 'allurings',
 'talons',
 'scenery',
 'grieve',
 'canes',
 'sneaks',
 'obstinate',
 'damndest',
 'gradations',
 'ventricles',
 '70',
 'besides',
 'sheepfold',
 'brother',
 'clock',
 'vows',
 'spark',
 'handspike',
 'exhaustion',
 'creditors',
 'curb',
 'wagon',
 'humbly',
 'benevolence',
 'absurdly',
 'excavating',
 'provide

In [49]:
type(prepare_sequence(list(vocab), word2index))

NoneType